# Supervisor Multi AI Agent Architecture
* Agents are dependent on the tasks provided by the supervisor

In [6]:
from typing import Annotated, TypedDict, List, Literal, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool

from langgraph.graph import END, START,StateGraph, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition, create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq

import random
from datetime import datetime

import os
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
# Defining supervisor class
class SupervisorState(MessagesState):
    """State for the supervisor multi-agent system"""
    next_agent: str = ""
    research_data: str = ""
    analysis: str = ""
    final_report: str = ""
    task_complete: bool = False
    current_task: str = ""

# creating supervisor chain
def create_supervisor_chain():
    """creates a supervisor decision chain"""
    supervisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """"You are a supervisor managing a team of agents:
            
            1. Researcher - Gathers information and data
            2. Analyst - Analyses the data and provides insights
            3. Writer - Creates reports and summaries

            Based on the current state and conversation , decide which agent should work next.
            If the task is completed, respond with 'Done'.

            Current state:
            - Has research data : {has_research}
            - Has analysis : {has_analysis}
            - Has report : {has_report}

            Respond with ONLY the agent name (researcher/analyst/writer) or 'Done'.
        """ ),
        ("human","{task}")
    ])

    return supervisor_prompt | llm

In [8]:
# creating supervisor node
def supervisor_agent(state: SupervisorState) -> Dict:
    """Supervisor decides which is the next agent to call using Groq LLM"""
    messages = state["messages"]

    task = messages[-1].content if messages else "No task"

    # Check whats been completed
    has_research = bool(state.get("research_data", ""))
    has_analysis = bool(state.get("analysis", ""))
    has_report = bool(state.get("final_report", ""))

    # Get LLM message
    chain = create_supervisor_chain()
    decision = chain.invoke({
        "task": task,
        "has_research": has_research,
        "has_analysis": has_analysis,
        "has_report": has_report
    })

    # Parse decision
    decision_text = decision.content.strip().lower()
    print(decision_text)

    # Determine next agent
    if "done" in decision_text or has_report:
        next_agent = "end"
        supervisor_msg = "Supervisor: All tasks have been completed! Great work team!"
    elif "researcher" in decision_text or not has_research:
        next_agent = "researcher"
        supervisor_msg = "Supervisor: Let's start with research. Assigning the task to Researcher...."
    elif "analyst" in decision_text or (has_research and not has_analysis):
        next_agent = "analyst"
        supervisor_msg = "Supervisor: Research done. Time for analysis. Assigning to Analyst..."
    elif "writer" in decision_text or (has_analysis and not has_report):
        next_agent = "writer"
        supervisor_msg = "Supervisor: Analysis done. Let's create the report. Assigning to writer"
    else:
        next_agent = "end"
        supervisor_msg = "Supervisor: Tasks seems complete"

    return {
        "messages": [AIMessage(content=supervisor_msg)],
        "next_agent":next_agent,
        "current_task": task
    }

In [ ]:
# Creating writer agent
def writer_agent(state: SupervisorState) -> Dict:
    """Writer agent uses Groq to write the final report"""

    research_data = state.get("research_data", "")
    analysis = state.get("analysis", "")
    task = state.get("current_task", "")

    # writing prompt
    writing_prompt = f"""As a professional writer, create an executive report based on:
    Task : {task}

    Research Findings:
    {research_data[:1000]}

    Analysis Findings:
    {analysis[:1000]}

    Create a well-structured report with:
    1. Executive Summary
    2. Key Findings
    3. Analysis & Insights
    4. Recommendations
    5. Conclusion

    Keep it professional and concise."""

    # Get report from LLM
    report_response = llm.invoke([HumanMessage(content = writing_prompt)])
    report = report_response.content

    # create final formatted report
    final_report = f"""
    FINAL REPORT
    {'='*50}
    Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}
    Topic: {task}
    {'='*50}

    {report}

    {'='*50}
    Report compiled by Multi-AI Agent system powered by Groq
    """
    return {
        "messages":[AIMessage(content=f"Writer: Report complete! See below for the full document.")],
        "final_report": final_report,
        "next_agent": "supervisor",
        "task_complete": True
    }
